# <b>GAN 실습</b>

* 논문 제목: Generative Adversarial Networks <b>(NIPS 2014)</b>
* 가장 기본적인 GAN 모델을 학습해보는 실습을 진행합니다.
* 학습 데이터셋: <b>MNIST</b> (1 X 28 X 28)

#### <b>필요한 라이브러리 불러오기</b>

* 실습을 위한 PyTorch 라이브러리를 불러옵니다.

In [ ]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

#### <b>생성자(Generator) 및 판별자(Discriminator) 모델 정의</b>

In [ ]:
latent_dim = 100

# 생성자(Generator) 클래스 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block) 정의
        def block(input_dim, output_dim):
            ##### TODO
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model = ##### TODO

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

In [ ]:
# 판별자(Discriminator) 클래스 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = ##### TODO

    # 이미지에 대한 판별 결과를 반환
    def forward(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output

#### <b>학습 데이터셋 불러오기</b>

* 학습을 위해 MNIST 데이터셋을 불러옵니다.

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)

100%|██████████| 9912422/9912422 [00:00<00:00, 401814161.05it/s]

Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 111851979.52it/s]


Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 127844783.29it/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3290246.76it/s]

Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import PIL
from torchvision.transforms import transforms

sample, label = train_dataset[0]
img = transforms.ToPILImage()(sample)

print(label)
display(img)

5


#### <b>모델 학습 및 샘플링</b>

* 학습을 위해 생성자와 판별자 모델을 초기화합니다.
* 적절한 하이퍼 파라미터를 설정합니다.

In [ ]:
device = 'cuda'
# 생성자(generator)와 판별자(discriminator) 초기화
generator = Generator()
discriminator = Discriminator()

# 손실 함수(loss function)
adversarial_loss = nn.BCELoss()

generator = generator.to(device)
discriminator = discriminator.to(device)
adversarial_loss = adversarial_loss.to(device)

# 학습률(learning rate) 설정
lr = 0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G = torch.optim.AdamW(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

* 모델을 학습하면서 주기적으로 샘플링하여 결과를 확인할 수 있습니다.

In [ ]:
import time

n_epochs = 200 # 학습의 횟수(epoch) 설정
sample_interval = 2000 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정
start_time = time.time()

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성
        real = torch.ones((imgs.size(0), 1)).to(device) # 진짜(real): 1
        fake = torch.zeros((imgs.size(0), 1)).to(device) # 가짜(fake): 0
        real_imgs = imgs.to(device)

        """ 생성자(generator)를 학습합니다. """
        optimizer_G.zero_grad()

        z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).to(device)
        generated_imgs = generator(z)

        ##### TODO: generator의 loss를 설계하세요.
        ##### HINT: 위에 제공된 adversarial_loss를 활용하세요.
        g_loss = ##
        #######################################################
        g_loss.backward()
        optimizer_G.step()

        """ 판별자(discriminator)를 학습합니다. """
        optimizer_D.zero_grad()

        ##### TODO: Discriminator의 loss를 설계하세요.
        ##### HINT: Generator를 학습시키지 않기 위해서,
        #####       generated_imgs는 generated_imgs.detach()로 사용해야 합니다.
        real_loss = ##
        fake_loss = ##
        d_loss = (real_loss + fake_loss) / 2
        ######################################################################
        d_loss.backward()
        optimizer_D.step()

        done = epoch * len(dataloader) + i
        if done % sample_interval == 0:
            # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
            save_image(generated_imgs.data[:25], f"result/{done}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

[Epoch 0/200] [D loss: 0.446626] [G loss: 0.894781] [Elapsed time: 14.01s]
[Epoch 1/200] [D loss: 0.633342] [G loss: 0.438454] [Elapsed time: 28.87s]
[Epoch 2/200] [D loss: 0.652284] [G loss: 0.389429] [Elapsed time: 42.35s]
[Epoch 3/200] [D loss: 0.457366] [G loss: 1.061799] [Elapsed time: 56.36s]
[Epoch 4/200] [D loss: 0.568809] [G loss: 0.602722] [Elapsed time: 70.09s]
[Epoch 5/200] [D loss: 0.492135] [G loss: 0.877076] [Elapsed time: 84.06s]
[Epoch 6/200] [D loss: 0.517846] [G loss: 0.897334] [Elapsed time: 98.21s]
[Epoch 7/200] [D loss: 0.618305] [G loss: 0.471885] [Elapsed time: 112.65s]
[Epoch 8/200] [D loss: 0.557629] [G loss: 0.764940] [Elapsed time: 127.11s]
[Epoch 9/200] [D loss: 0.395843] [G loss: 1.120053] [Elapsed time: 141.43s]
[Epoch 10/200] [D loss: 0.553836] [G loss: 0.585178] [Elapsed time: 155.83s]
[Epoch 11/200] [D loss: 0.527404] [G loss: 0.756308] [Elapsed time: 170.55s]
[Epoch 12/200] [D loss: 0.778111] [G loss: 0.308285] [Elapsed time: 186.88s]
[Epoch 13/200] [

* 생성된 이미지 예시를 출력합니다.

In [ ]:
from IPython.display import Image

Image('92000.png')

# Multihead Attention Implementation

In [ ]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim=768, n_heads=8, device='cuda'):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원


        #####TODO: Q,K,V를 만들기 위한 세 개의 linear module 구성하기
        #####HINT: 해당 class의 input / out dim은 모두 hidden dim과 동일하다고 가정합니다.
        self.fc_q = # query 값에 적용될 FC 레이어
        self.fc_k = # key 값에 적용될 FC 레이어
        self.fc_v = # value 값에 적용될 FC 레이어
        #################################################################################

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value):

        batch_size = query.shape[0]

        # query: [batch_size, query_len, hidden_dim]
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        ##### TODO: Q,K,V 생성하기
        Q = #
        K = #
        V = #
        #########################
        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]


        ##### TODO: hidden_dim → n_heads X head_dim 형태로 변형
        ##### HINT: .view와 .permute를 사용해보세요!
        ##### Q: [batch_size, query_len, hidden_dim] => [batch_size, n_heads, query_len, head_dim]
        ##### K: [batch_size, key_len, hidden_dim]   => [batch_size, n_heads, key_len, head_dim]
        ##### V: [batch_size, value_len, hidden_dim] => [batch_size, n_heads, value_len, hidden_dim]
        Q = ##
        K = ##
        V = ##
        ############################################################################################


        ##### TODO: Attention Energy(Similiarity) / Score 계산
        ##### HINT: 해당 Matrix는 4차원입니다. torch의 다차원 행렬곱 (torch.matmul)을 사용하고,
        #####       .permute를 사용하여 행렬이 Transpose되도록 적절히 차원의 순서를 변경하세요.
        energy = ##
        ##### HINT: 어떤 축을 기준으로 softmax를 해야할까요?
        ##### score: [batch_size, n_heads, query_len, key_len]
        score  = ##
        ######################################################################################

        #  Scaled Dot-Product Attention을 계산
        x = torch.matmul(score, V)
        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous()
        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim)
        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)
        # x: [batch_size, query_len, hidden_dim]

        return x, score

## 정답

In [ ]:
answer = torch.Tensor([[[-0.1632, -0.3923, -0.1880,  0.0097,  0.0657, -0.0539,  0.3968,
           0.6771, -0.5740, -0.1323, -0.3527, -0.2200, -0.2332, -0.9086,
           0.0321, -0.0847, -0.0698,  0.0040, -0.1179, -0.0800, -0.1619,
           0.4520, -0.5898, -0.2019, -0.0072, -0.6319,  0.0394,  0.8672,
          -0.0165,  0.0392,  0.5124, -0.9029, -0.1177,  0.4774,  0.0482,
           0.2276, -0.4341, -0.9641, -0.6959, -0.0606, -0.2401, -0.3868,
          -0.1382,  0.1295, -0.5624, -0.6337, -0.1560,  0.0847,  0.2148,
           0.3581,  0.0647, -0.0222, -0.1514,  0.0068,  0.5101, -0.0324,
           0.1693,  0.3731, -0.1654,  0.1863, -0.0343,  0.2678, -0.1978,
           0.1282,  0.0679,  0.2835, -0.0860, -0.1652,  0.9319,  0.4850,
           0.3394,  0.1817, -0.0627, -0.0060,  0.2428,  0.1568,  0.0638,
          -0.1312, -0.3751, -0.4833,  0.7383, -0.3752,  0.1200,  0.0292,
           0.0550, -0.2755, -0.1751,  0.3529, -0.2790,  0.0943,  0.2216,
           0.1300, -0.1737, -0.2964, -0.3078,  0.0427, -0.1384,  0.0989,
          -0.2037,  0.2713, -0.4173,  0.3733, -0.0172,  0.1233,  0.5338,
          -0.2690, -0.3113, -0.1544,  0.1079,  0.1611,  0.1053,  0.1493,
           0.1799, -0.0348,  0.0443, -0.0207,  0.1240, -0.2744, -0.3715,
           0.0812, -0.3096,  0.3574, -0.6020, -0.0387,  0.2096, -0.3036,
           0.5056, -0.1336],
         [-0.1632, -0.3923, -0.1880,  0.0097,  0.0657, -0.0539,  0.3968,
           0.6771, -0.5740, -0.1323, -0.3527, -0.2200, -0.2332, -0.9086,
           0.0321, -0.0847, -0.0698,  0.0040, -0.1179, -0.0800, -0.1619,
           0.4520, -0.5898, -0.2019, -0.0072, -0.6319,  0.0394,  0.8672,
          -0.0165,  0.0392,  0.5124, -0.9029, -0.1177,  0.4774,  0.0482,
           0.2276, -0.4341, -0.9641, -0.6959, -0.0606, -0.2401, -0.3868,
          -0.1382,  0.1295, -0.5624, -0.6337, -0.1560,  0.0847,  0.2148,
           0.3581,  0.0647, -0.0222, -0.1514,  0.0068,  0.5101, -0.0324,
           0.1693,  0.3731, -0.1654,  0.1863, -0.0343,  0.2678, -0.1978,
           0.1282,  0.0679,  0.2835, -0.0860, -0.1652,  0.9319,  0.4850,
           0.3394,  0.1817, -0.0627, -0.0060,  0.2428,  0.1568,  0.0638,
          -0.1312, -0.3751, -0.4833,  0.7383, -0.3752,  0.1200,  0.0292,
           0.0550, -0.2755, -0.1751,  0.3529, -0.2790,  0.0943,  0.2216,
           0.1300, -0.1737, -0.2964, -0.3078,  0.0427, -0.1384,  0.0989,
          -0.2037,  0.2713, -0.4173,  0.3733, -0.0172,  0.1233,  0.5338,
          -0.2690, -0.3113, -0.1544,  0.1079,  0.1611,  0.1053,  0.1493,
           0.1799, -0.0348,  0.0443, -0.0207,  0.1240, -0.2744, -0.3715,
           0.0812, -0.3096,  0.3574, -0.6020, -0.0387,  0.2096, -0.3036,
           0.5056, -0.1336],
         [-0.1632, -0.3923, -0.1880,  0.0097,  0.0657, -0.0539,  0.3968,
           0.6771, -0.5740, -0.1323, -0.3527, -0.2200, -0.2332, -0.9086,
           0.0321, -0.0847, -0.0698,  0.0040, -0.1179, -0.0800, -0.1619,
           0.4520, -0.5898, -0.2019, -0.0072, -0.6319,  0.0394,  0.8672,
          -0.0165,  0.0392,  0.5124, -0.9029, -0.1177,  0.4774,  0.0482,
           0.2276, -0.4341, -0.9641, -0.6959, -0.0606, -0.2401, -0.3868,
          -0.1382,  0.1295, -0.5624, -0.6337, -0.1560,  0.0847,  0.2148,
           0.3581,  0.0647, -0.0222, -0.1514,  0.0068,  0.5101, -0.0324,
           0.1693,  0.3731, -0.1654,  0.1863, -0.0343,  0.2678, -0.1978,
           0.1282,  0.0679,  0.2835, -0.0860, -0.1652,  0.9319,  0.4850,
           0.3394,  0.1817, -0.0627, -0.0060,  0.2428,  0.1568,  0.0638,
          -0.1312, -0.3751, -0.4833,  0.7383, -0.3752,  0.1200,  0.0292,
           0.0550, -0.2755, -0.1751,  0.3529, -0.2790,  0.0943,  0.2216,
           0.1300, -0.1737, -0.2964, -0.3078,  0.0427, -0.1384,  0.0989,
          -0.2037,  0.2713, -0.4173,  0.3733, -0.0172,  0.1233,  0.5338,
          -0.2690, -0.3113, -0.1544,  0.1079,  0.1611,  0.1053,  0.1493,
           0.1799, -0.0348,  0.0443, -0.0207,  0.1240, -0.2744, -0.3715,
           0.0812, -0.3096,  0.3574, -0.6020, -0.0387,  0.2096, -0.3036,
           0.5056, -0.1336]]]).to(device)

## 채점

In [ ]:
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

# Check MultiHeadAttention
rand_input = torch.ones((1,3,128)).to(device)
MHA = MultiHeadAttentionLayer(hidden_dim=128).to(device)
MHA_output = MHA(rand_input, rand_input, rand_input)
if -0.002 < torch.sum(MHA_output[0] - answer) < 0.002:
    print("sanity check passed")
else: print("sanity check failed")

sanity check passed
